# European Bond Option Pricing - HW Model Jamshidian

Jamshidian's model is a fast way to price European bond options in the HW model. Her we value an option on a coupon paying bond using the Hull-White model and comparing the tree and the Jamshidian model for European Bond Options

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
from financepy.utils import *
from financepy.market.curves import *
from financepy.models.hw_tree import HWTree, FinHWEuropeanCalcType
from financepy.products.bonds import *

####################################################################
# FINANCEPY BETA Version 0.260 - This build:  22 Nov 2022 at 13:08 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Set up Discount Curve

In [3]:
settlement_date = Date(1, 12, 2019)

Set up discount curve

In [4]:
rate = 0.05
dcType = DayCountTypes.THIRTY_360_BOND
fixedFreq = FrequencyTypes.SEMI_ANNUAL
discount_curve = DiscountCurveFlat(settlement_date, rate, fixedFreq, dcType)

## Set up the Bond Option

First create the bond

In [5]:
issue_date = Date(1,12,2018)
maturity_date = issue_date.add_tenor("10Y")
coupon = 0.06
frequencyType = FrequencyTypes.SEMI_ANNUAL
accrual_type = DayCountTypes.THIRTY_360_BOND
bond = Bond(issue_date, maturity_date, coupon, frequencyType, accrual_type)

Let's first price the bond on the libor curve

In [6]:
cp = bond.clean_price_from_discount_curve(settlement_date, discount_curve)
dp = bond.dirty_price_from_discount_curve(settlement_date, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   107.177
Fixed Income Full  Price:   110.177


It's par as the discount rate is the same as the swap rate and the swap cash flows are 30/360 so like bond flows.

In [7]:
print(bond.coupon_dates(settlement_date))

 01-DEC-2019         3.00 
 01-JUN-2020         3.00 
 01-DEC-2020         3.00 
 01-JUN-2021         3.00 
 01-DEC-2021         3.00 
 01-JUN-2022         3.00 
 01-DEC-2022         3.00 
 01-JUN-2023         3.00 
 01-DEC-2023         3.00 
 01-JUN-2024         3.00 
 01-DEC-2024         3.00 
 01-JUN-2025         3.00 
 01-DEC-2025         3.00 
 01-JUN-2026         3.00 
 01-DEC-2026         3.00 
 01-JUN-2027         3.00 
 01-DEC-2027         3.00 
 01-JUN-2028         3.00 
 01-DEC-2028       103.00 



## Create the Bond Options

Then define and create the options

In [10]:
expiry_date = settlement_date.add_tenor("18m")
strike_price = 105.0
face_amount = 100.0

In [11]:
europeanCallBondOption = BondOption(bond, expiry_date, strike_price,
                                       face_amount, OptionTypes.EUROPEAN_CALL)

In [12]:
europeanPutBondOption = BondOption(bond, expiry_date, strike_price,
                                      face_amount, OptionTypes.EUROPEAN_PUT)

Let's look at the forward price to calculate intrinsic.

In [13]:
cp = bond.clean_price_from_discount_curve(expiry_date, discount_curve)
dp = bond.dirty_price_from_discount_curve(expiry_date, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   106.191
Fixed Income Full  Price:   109.191


In [14]:
df = discount_curve.df(expiry_date)

In [15]:
CallPx = max(cp - strike_price,0) * df

In [16]:
PutPx = max(strike_price - cp,0) * df

In [17]:
print(CallPx, PutPx)

1.1056729769751705 0.0


## Setting up Models

Set up all three bond option model

In [18]:
sigma = 0.01
a = 0.1
num_steps = 200
modelJamshidian = HWTree(sigma, a, num_steps, FinHWEuropeanCalcType.JAMSHIDIAN)
modelExpiryOnly = HWTree(sigma, a, num_steps, FinHWEuropeanCalcType.EXPIRY_ONLY)
modelExpiryTree = HWTree(sigma, a, num_steps, FinHWEuropeanCalcType.EXPIRY_TREE)

## Comparing Jamshidian and Tree Implementations

In [19]:
ec1 = europeanCallBondOption.value(settlement_date, discount_curve, modelJamshidian)
ec2 = europeanCallBondOption.value(settlement_date, discount_curve, modelExpiryOnly)
ec3 = europeanCallBondOption.value(settlement_date, discount_curve, modelExpiryTree)
print("European Call Jamshidian  Value: %9.5f" % ec1)
print("European Call Expiry Only Value: %9.5f" % ec2)
print("European Call Expiry Tree Value: %9.5f" % ec3)

European Call Jamshidian  Value:   4.47008
European Call Expiry Only Value:   4.47687
European Call Expiry Tree Value:   2.58983


In [20]:
ep1 = europeanPutBondOption.value(settlement_date, discount_curve, modelJamshidian)
ep2 = europeanPutBondOption.value(settlement_date, discount_curve, modelExpiryOnly)
ep3 = europeanPutBondOption.value(settlement_date, discount_curve, modelExpiryTree)
print("European Put Jamshidian Value:  %9.5f" % ep1)
print("European Put Expiry Only Value: %9.5f" % ep2)
print("European Put Expiry Tree Value: %9.5f" % ep3)

European Put Jamshidian Value:    0.60540
European Put Expiry Only Value:   0.60916
European Put Expiry Tree Value:   1.50772


## Timing Comparisons

In [21]:
%timeit ep1 = europeanPutBondOption.value(settlement_date, discount_curve, modelJamshidian)

936 µs ± 24.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
%timeit ep2 = europeanPutBondOption.value(settlement_date, discount_curve, modelExpiryOnly)

2.94 ms ± 71.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit ep3 = europeanPutBondOption.value(settlement_date, discount_curve, modelExpiryTree)

515 µs ± 5.41 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Interestingly, the expiry tree is fastest. Possibly because the Jamshidian main loop has not been Numba'd

## Strike Dependency

In [24]:
strikes = np.linspace(70,130,31)

In [25]:
for K in strikes:
    europeanCallBondOption = BondOption(bond, expiry_date, K, face_amount, OptionTypes.EUROPEAN_CALL)
    ec1 = europeanCallBondOption.value(settlement_date, discount_curve, modelJamshidian)
    ec2 = europeanCallBondOption.value(settlement_date, discount_curve, modelExpiryOnly)
    ec3 = europeanCallBondOption.value(settlement_date, discount_curve, modelExpiryTree)    
    print("K: %9.5f JAMSHIDIAN: %9.5f  EXPIRY_ONLY: %9.5f  EXPIRY_TREE: %9.5f " %(K, ec1, ec2, ec3))

K:  70.00000 JAMSHIDIAN:  36.36225  EXPIRY_ONLY:  36.39060  EXPIRY_TREE:  33.60499 
K:  72.00000 JAMSHIDIAN:  34.50522  EXPIRY_ONLY:  34.53215  EXPIRY_TREE:  31.74654 
K:  74.00000 JAMSHIDIAN:  32.64820  EXPIRY_ONLY:  32.67370  EXPIRY_TREE:  29.88809 
K:  76.00000 JAMSHIDIAN:  30.79118  EXPIRY_ONLY:  30.81525  EXPIRY_TREE:  28.02964 
K:  78.00000 JAMSHIDIAN:  28.93417  EXPIRY_ONLY:  28.95680  EXPIRY_TREE:  26.17119 
K:  80.00000 JAMSHIDIAN:  27.07715  EXPIRY_ONLY:  27.09835  EXPIRY_TREE:  24.31274 
K:  82.00000 JAMSHIDIAN:  25.22014  EXPIRY_ONLY:  25.23990  EXPIRY_TREE:  22.45429 
K:  84.00000 JAMSHIDIAN:  23.36313  EXPIRY_ONLY:  23.38145  EXPIRY_TREE:  20.59584 
K:  86.00000 JAMSHIDIAN:  21.50613  EXPIRY_ONLY:  21.52300  EXPIRY_TREE:  18.73740 
K:  88.00000 JAMSHIDIAN:  19.64913  EXPIRY_ONLY:  19.66455  EXPIRY_TREE:  16.87905 
K:  90.00000 JAMSHIDIAN:  17.79216  EXPIRY_ONLY:  17.80613  EXPIRY_TREE:  15.02108 
K:  92.00000 JAMSHIDIAN:  15.93540  EXPIRY_ONLY:  15.94794  EXPIRY_TREE:  13

All three methods are consistent.

Copyright (c) Dominic O'Kane 2020